In [1]:
# ===========================================
# 0. Setup
# ===========================================
import os
import json
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler

BASE_02 = "/workspace/data/02/players"
BASE_03 = "/workspace/data/03/dynamic"
BASE_03_STATIC = "/workspace/data/03/static"  # ★ static 出力
os.makedirs(BASE_03, exist_ok=True)

DYNAMIC_CONFIG = {
    "seq_len": 30,
    "break_days": 30,
    "hidden_dim": 64,
    "latent_dim": 32,
    "num_layers": 2,
    "dropout": 0.1,
    "epochs": 40,
    "batch_size": 32,
    "reset_days": 7,

    # ★ S1: sigma閾値をやめて分位点閾値にする（頑健）
    "threshold_q": 0.99,

    # 学習に使う最低シーケンス数（少なすぎると不安定）
    "min_train_seqs": 50,
    "random_state": 42,
}

META_EXCLUDE = ["athlete_id", "athlete_name", "date_", "md_offset", "md_phase", "is_match_day"]



In [2]:
# ===========================================
# 1. Model & Dataset
# ===========================================
class SequenceDataset(Dataset):
    def __init__(self, seq_array):
        self.seq_array = seq_array

    def __len__(self):
        return len(self.seq_array)

    def __getitem__(self, idx):
        return torch.tensor(self.seq_array[idx], dtype=torch.float32)


class LSTMAutoEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, num_layers, dropout):
        super().__init__()

        self.encoder = nn.LSTM(
            input_dim, hidden_dim,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )
        self.encoder_fc = nn.Linear(hidden_dim, latent_dim)

        self.decoder_fc = nn.Linear(latent_dim, hidden_dim)
        self.decoder = nn.LSTM(
            hidden_dim, input_dim,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )

    def forward(self, x):
        enc_out, _ = self.encoder(x)
        h_last = enc_out[:, -1, :]
        z = self.encoder_fc(h_last)

        dec_init = self.decoder_fc(z).unsqueeze(1)
        dec_init = dec_init.repeat(1, x.size(1), 1)

        dec_out, _ = self.decoder(dec_init)
        return dec_out

In [3]:
# ===========================================
# 2. Dynamic preprocessing
# ===========================================
def add_season_block(df_dyn: pd.DataFrame, break_days: int) -> pd.DataFrame:
    df_dyn = df_dyn.sort_values("date_").reset_index(drop=True)
    dates = pd.to_datetime(df_dyn["date_"])
    gap_days = dates.diff().dt.days
    season_block = (gap_days > break_days).cumsum().fillna(0).astype(int)
    df_dyn["season_block"] = season_block.values
    return df_dyn


def add_season_reset_zone(df_dyn: pd.DataFrame, reset_days: int) -> pd.DataFrame:
    """
    season_block の切り替わり直後 reset_days を 1 にする
    """
    df_dyn = df_dyn.copy()
    df_dyn["season_reset_zone"] = 0
    for b, sub in df_dyn.groupby("season_block"):
        if b == 0:
            continue
        start = pd.to_datetime(sub["date_"]).min()
        end = start + pd.Timedelta(days=reset_days - 1)
        mask = (pd.to_datetime(df_dyn["date_"]) >= start) & (pd.to_datetime(df_dyn["date_"]) <= end)
        df_dyn.loc[mask, "season_reset_zone"] = 1
    return df_dyn


def load_static_flags(athlete_id: str) -> pd.DataFrame:
    """
    static_labels.parquet から date_ ごとの static_anomaly を読む
    無ければ None
    """
    p = os.path.join(BASE_03_STATIC, str(athlete_id), "static_labels.parquet")
    if not os.path.exists(p):
        return None
    s = pd.read_parquet(p)
    s["date_"] = pd.to_datetime(s["date_"])
    return s[["date_", "static_anomaly"]]


def scale_per_block(df_dyn: pd.DataFrame, feature_cols: list):
    """
    season_block ごとに StandardScaler でスケール
    + backend用に scaler params を保存できるように返す
    """
    X_raw = df_dyn[feature_cols].astype(float).values
    X_scaled = np.zeros_like(X_raw)

    scaler_params = {}  # block -> {mean, scale}
    for b in sorted(df_dyn["season_block"].unique()):
        mask = (df_dyn["season_block"].values == b)
        scaler = StandardScaler()
        X_scaled[mask] = scaler.fit_transform(X_raw[mask])

        scaler_params[int(b)] = {
            "mean": scaler.mean_.tolist(),
            "scale": scaler.scale_.tolist(),
        }

    return X_scaled, scaler_params


def build_sequences(X_scaled: np.ndarray, dates: np.ndarray, seq_len: int):
    """
    全データから推論用のシーケンスを作る（seq_datesは窓末尾日）
    """
    seq_list, seq_dates, end_idx = [], [], []
    if len(X_scaled) < seq_len:
        return None, None, None

    for i in range(seq_len - 1, len(X_scaled)):
        seq_list.append(X_scaled[i - seq_len + 1:i + 1])
        seq_dates.append(dates[i])
        end_idx.append(i)

    return np.stack(seq_list), np.array(seq_dates), np.array(end_idx)

In [4]:
# ===========================================
# 3. Dynamic anomaly (single player)
# ===========================================
def run_dynamic_anomaly_single_player(df_dyn: pd.DataFrame, athlete_id: str, config: dict):

    df_dyn = df_dyn.copy()
    df_dyn["date_"] = pd.to_datetime(df_dyn["date_"])
    df_dyn = add_season_block(df_dyn, break_days=config["break_days"])
    df_dyn = add_season_reset_zone(df_dyn, reset_days=config["reset_days"])

    # ★ S2: static flags merge（無ければ 0 扱い）
    static_flags = load_static_flags(athlete_id)
    if static_flags is not None:
        df_dyn = df_dyn.merge(static_flags, on="date_", how="left")
        df_dyn["static_anomaly"] = df_dyn["static_anomaly"].fillna(0).astype(int)
    else:
        df_dyn["static_anomaly"] = 0

    # 特徴量列
    feature_cols = [c for c in df_dyn.columns if c not in META_EXCLUDE + ["season_block", "season_reset_zone", "static_anomaly"]]
    feature_cols = [c for c in feature_cols if pd.api.types.is_numeric_dtype(df_dyn[c])]

    if len(feature_cols) == 0:
        return None, None

    # 欠損：まずは median で埋める（AEの入力がNaNだと壊れる）
    impute_median = df_dyn[feature_cols].median()
    df_dyn[feature_cols] = df_dyn[feature_cols].fillna(impute_median)

    # ブロックごとスケール + params保存
    X_scaled, scaler_params = scale_per_block(df_dyn, feature_cols)
    dates = df_dyn["date_"].values

    # 推論用：全日でシーケンス生成
    seq_all, seq_dates, end_idx = build_sequences(X_scaled, dates, seq_len=config["seq_len"])
    if seq_all is None or len(seq_all) < 20:
        return None, None

    # ★ 学習用：end_idx に対応する日付のフラグで除外（reset_zone, static_anomaly）
    end_rows = df_dyn.iloc[end_idx].reset_index(drop=True)
    train_mask = (end_rows["season_reset_zone"].values == 0) & (end_rows["static_anomaly"].values == 0)

    seq_train = seq_all[train_mask]
    if len(seq_train) < config.get("min_train_seqs", 50):
        # 学習データが少なすぎる場合はスキップ
        return None, None

    # Dataset / Loader
    dataset = SequenceDataset(seq_train)
    train_loader = DataLoader(dataset, batch_size=config["batch_size"], shuffle=True)

    # device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # reproducibility (軽く)
    torch.manual_seed(config.get("random_state", 42))
    np.random.seed(config.get("random_state", 42))

    model = LSTMAutoEncoder(
        input_dim=seq_all.shape[2],
        hidden_dim=config["hidden_dim"],
        latent_dim=config["latent_dim"],
        num_layers=config["num_layers"],
        dropout=config["dropout"]
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()

    # ---- train ----
    for _ in range(config["epochs"]):
        model.train()
        for x in train_loader:
            x = x.to(device)
            optimizer.zero_grad()
            recon = model(x)
            loss = criterion(recon, x)
            loss.backward()
            optimizer.step()

    # ---- reconstruction error for ALL sequences ----
    model.eval()
    errors = []
    with torch.no_grad():
        for i in range(len(seq_all)):
            x = torch.tensor(seq_all[i:i+1], dtype=torch.float32).to(device)
            recon = model(x)
            err = ((recon - x) ** 2).mean().item()
            errors.append(err)
    errors = np.array(errors)

    df_seq = pd.DataFrame({
        "date_": pd.to_datetime(seq_dates),
        "dyn_error": errors,
    })

    # join flags (reset/static) for the end-date rows
    df_flags = end_rows[["date_", "season_block", "season_reset_zone", "static_anomaly"]].copy()
    df_flags["date_"] = pd.to_datetime(df_flags["date_"])
    df_seq = df_seq.merge(df_flags, on="date_", how="left")

    # ★ S1: threshold by quantile on VALID errors (not reset, not static anomaly)
    valid_err = df_seq.loc[
        (df_seq["season_reset_zone"] == 0) & (df_seq["static_anomaly"] == 0),
        "dyn_error"
    ]

    thr = float(np.quantile(valid_err.values, config["threshold_q"]))
    df_seq["dyn_thr"] = thr

    df_seq["dyn_anomaly"] = (
        (df_seq["dyn_error"] > thr) &
        (df_seq["season_reset_zone"] == 0)
    ).astype(int)

    # 持続指標（フロントで強い）
    df_seq = df_seq.sort_values("date_").reset_index(drop=True)
    # streak: 連続異常日数
    streak = []
    cur = 0
    for a in df_seq["dyn_anomaly"].values:
        if a == 1:
            cur += 1
        else:
            cur = 0
        streak.append(cur)
    df_seq["dyn_streak"] = streak

    # ★ S3: artifacts
    artifacts = {
        "feature_cols": feature_cols,
        "impute_median": {k: float(v) if pd.notna(v) else None for k, v in impute_median.to_dict().items()},
        "scaler_params_per_block": scaler_params,
        "threshold_q": config["threshold_q"],
        "dyn_thr": thr,
        "config": config,
    }

    return df_seq, artifacts


In [5]:
# ===========================================
# 4. Run for ALL players
# ===========================================
athlete_dirs = sorted([d for d in os.listdir(BASE_02) if os.path.isdir(os.path.join(BASE_02, d))])
print("対象選手数:", len(athlete_dirs))

for athlete_id in athlete_dirs:

    dyn_path = os.path.join(BASE_02, athlete_id, f"{athlete_id}_dynamic.parquet")
    if not os.path.exists(dyn_path):
        continue

    df_dyn = pd.read_parquet(dyn_path)

    res, artifacts = run_dynamic_anomaly_single_player(df_dyn, athlete_id=str(athlete_id), config=DYNAMIC_CONFIG)
    if res is None:
        continue

    res["athlete_id"] = athlete_id
    res["method"] = "LSTM_AE"
    res["params"] = json.dumps(DYNAMIC_CONFIG, ensure_ascii=False)

    out_dir = os.path.join(BASE_03, str(athlete_id))
    os.makedirs(out_dir, exist_ok=True)

    out_path = os.path.join(out_dir, "dynamic_labels.parquet")
    res.to_parquet(out_path, index=False)

    art_path = os.path.join(out_dir, "dynamic_artifacts.json")
    with open(art_path, "w") as f:
        json.dump(artifacts, f, ensure_ascii=False)

    print(f"✅ saved dynamic anomaly → {athlete_id}")

print("🎉 Dynamic anomaly (all players) completed")


対象選手数: 30
✅ saved dynamic anomaly → 121b05df-f5f6-4029-92a7-5420dea45e4d
✅ saved dynamic anomaly → 13bb34b4-8c38-4c86-86b8-bbe8574988c8
✅ saved dynamic anomaly → 15d36f96-6a91-4787-96f8-5fdf8565006b
✅ saved dynamic anomaly → 223a7cbc-a76b-4e36-ab5c-215fc9492e84
✅ saved dynamic anomaly → 3ded61ff-c67b-4776-a1ef-5050bb5c7fd3
✅ saved dynamic anomaly → 44eea4b6-3614-4ca2-b8d7-098b6120c1fb
✅ saved dynamic anomaly → 45f771a4-dab1-4e2a-8f92-cd219b677ab9
✅ saved dynamic anomaly → 4759d9d8-9e0e-44b8-9c70-874bf974c4ba
✅ saved dynamic anomaly → 68341b43-2561-4972-95f4-341f8530e023
✅ saved dynamic anomaly → 6eda50d0-970c-44ab-b470-de9ebc71ae52
✅ saved dynamic anomaly → 714e5fd4-609c-4e96-bea6-b7f3966bf681
✅ saved dynamic anomaly → 7995a06b-bb0d-4343-bc18-cbdf623d4640
✅ saved dynamic anomaly → 83761ac5-fbd3-4422-ba30-b6b4915da945
✅ saved dynamic anomaly → 83a85906-44bd-4976-8906-53faed1684f3
✅ saved dynamic anomaly → 854ad249-d0e7-4e32-bdab-adde8740c980
✅ saved dynamic anomaly → 8fdd27ba-fabe-4df0-

In [6]:
# ===========================================
# Dynamic anomaly : 最小チェックセット
# ===========================================
import os
import pandas as pd

BASE_03 = "/workspace/data/03/dynamic"

# -------------------------------------------
# 1. 全選手の dynamic_labels を読み込み
# -------------------------------------------
dfs = []

for athlete_id in os.listdir(BASE_03):
    p = os.path.join(BASE_03, athlete_id, "dynamic_labels.parquet")
    if os.path.exists(p):
        df = pd.read_parquet(p)
        df["athlete_id"] = athlete_id
        dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)

print("✅ Loaded dynamic results")
print("Total rows:", len(df_all))
print("Players   :", df_all["athlete_id"].nunique())


# -------------------------------------------
# 2. 全体の異常率（まず壊れてないか）
# -------------------------------------------
print("\n=== [1] Overall dyn_anomaly counts ===")
print(df_all["dyn_anomaly"].value_counts())
print("Overall anomaly rate:",
      df_all["dyn_anomaly"].mean().round(4))


# -------------------------------------------
# 3. 選手別の異常率分布（staticと同じ）
# -------------------------------------------
player_rates = (
    df_all
    .groupby("athlete_id")["dyn_anomaly"]
    .mean()
)

print("\n=== [2] Player-wise anomaly rate (describe) ===")
print(player_rates.describe())


# -------------------------------------------
# 4. dyn_streak の最大値（連続異常の確認）
# -------------------------------------------
max_streak = (
    df_all
    .groupby("athlete_id")["dyn_streak"]
    .max()
    .sort_values(ascending=False)
)

print("\n=== [3] Max dyn_streak per player (TOP 5) ===")
print(max_streak.head(5))


# -------------------------------------------
# 5. dyn_thr の分布（極端値チェック）
# -------------------------------------------
print("\n=== [4] dyn_thr distribution ===")
print(df_all["dyn_thr"].describe())


# -------------------------------------------
# 6. 参考：異常日のみを見る（目視用）
# -------------------------------------------
print("\n=== [5] Sample anomaly rows ===")
display(
    df_all[df_all["dyn_anomaly"] == 1]
    .sort_values(["athlete_id", "date_"])
    .head(10)
)


✅ Loaded dynamic results
Total rows: 10324
Players   : 30

=== [1] Overall dyn_anomaly counts ===
dyn_anomaly
0    10200
1      124
Name: count, dtype: int64
Overall anomaly rate: 0.012

=== [2] Player-wise anomaly rate (describe) ===
count    30.000000
mean      0.012721
std       0.002364
min       0.009756
25%       0.010639
50%       0.012180
75%       0.014406
max       0.018519
Name: dyn_anomaly, dtype: float64

=== [3] Max dyn_streak per player (TOP 5) ===
athlete_id
714e5fd4-609c-4e96-bea6-b7f3966bf681    6
d5f7cf9f-2d74-4bed-96a5-0c2650a2cdfc    5
4759d9d8-9e0e-44b8-9c70-874bf974c4ba    5
9a1d2d89-e1f6-4984-8a33-aa7f2473e5d7    5
121b05df-f5f6-4029-92a7-5420dea45e4d    4
Name: dyn_streak, dtype: int64

=== [4] dyn_thr distribution ===
count    10324.000000
mean         2.424251
std          1.292441
min          1.061228
25%          1.337830
50%          2.118569
75%          2.623135
max          4.980005
Name: dyn_thr, dtype: float64

=== [5] Sample anomaly rows ===


,date_,dyn_error,season_block,season_reset_zone,static_anomaly,dyn_thr,dyn_anomaly,dyn_streak,athlete_id,method,params
2168,2025-01-20,5.259851,2,0,1,4.980005,1,1,121b05df-f5f6-4029-92a7-5420dea45e4d,LSTM_AE,"{""seq_len"": 30, ""break_days"": 30, ""hidden_dim""..."
2169,2025-01-21,5.243083,2,0,0,4.980005,1,2,121b05df-f5f6-4029-92a7-5420dea45e4d,LSTM_AE,"{""seq_len"": 30, ""break_days"": 30, ""hidden_dim""..."
2170,2025-01-24,5.263999,2,0,0,4.980005,1,3,121b05df-f5f6-4029-92a7-5420dea45e4d,LSTM_AE,"{""seq_len"": 30, ""break_days"": 30, ""hidden_dim""..."
2171,2025-01-25,5.129557,2,0,0,4.980005,1,4,121b05df-f5f6-4029-92a7-5420dea45e4d,LSTM_AE,"{""seq_len"": 30, ""break_days"": 30, ""hidden_dim""..."
2184,2025-02-11,4.982769,2,0,0,4.980005,1,1,121b05df-f5f6-4029-92a7-5420dea45e4d,LSTM_AE,"{""seq_len"": 30, ""break_days"": 30, ""hidden_dim""..."
2185,2025-02-12,5.043195,2,0,0,4.980005,1,2,121b05df-f5f6-4029-92a7-5420dea45e4d,LSTM_AE,"{""seq_len"": 30, ""break_days"": 30, ""hidden_dim""..."
2186,2025-02-13,5.160212,2,0,0,4.980005,1,3,121b05df-f5f6-4029-92a7-5420dea45e4d,LSTM_AE,"{""seq_len"": 30, ""break_days"": 30, ""hidden_dim""..."
8239,2023-02-22,1.061552,0,0,0,1.061228,1,1,13bb34b4-8c38-4c86-86b8-bbe8574988c8,LSTM_AE,"{""seq_len"": 30, ""break_days"": 30, ""hidden_dim""..."
8443,2023-11-30,1.069523,0,0,0,1.061228,1,1,13bb34b4-8c38-4c86-86b8-bbe8574988c8,LSTM_AE,"{""seq_len"": 30, ""break_days"": 30, ""hidden_dim""..."
8444,2023-12-01,1.127281,0,0,0,1.061228,1,2,13bb34b4-8c38-4c86-86b8-bbe8574988c8,LSTM_AE,"{""seq_len"": 30, ""break_days"": 30, ""hidden_dim""..."
